In this notebook, we will load the LLLM GeneZC/MiniChat-3B to see how it performs. Here are the main steps:

Step 1. Transform user's input to vector with Sentence Transform.

Step 2. Compute the similarity between vectorized users' input and vectorized "movie recommendation": 

    - If the similarity is high, we then response based on model's knowledge and then perform vector search based on additional information from the response.

    - Otherwise, we will let the model answer with its own knowledge.

We note that suitable prompts are very important to extract useful information and generate meaningful answer.

In [1]:
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import sentencepiece

/home/nhannguyen/.local/share/virtualenvs/movie-recommendation-LLM-HHW9OGcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the model with bits and bytes config
model = "GeneZC/MiniChat-3B"
# model = "mistralai/Mistral-7B-Instruct-v0.1"
# model = "openai-community/gpt2-xl"

MODEL_NAME = model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # device_map="cpu",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# PEFT wrapper the model for training / fine-tuning
model = prepare_model_for_kbit_training(model)

# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
# Load sentence transformer model
from sentence_transformers import SentenceTransformer
filter_model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
# Get vector from an input text by Sentence Transformer
def get_vector(text):
    text_vector = filter_model.encode(text)
    return torch.tensor(text_vector)

In [5]:
# Vectorization of "movie recommendation"
valid_query = "movie recommendation"
valid_vector = get_vector(valid_query)
valid_vector.shape

torch.Size([384])

In [6]:
# Generate response based on a suitable prompt
def bot_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate answer
    output_sequences = model.generate(
        **inputs,
        max_length=1000,
        num_return_sequences=1,
        temperature=0.1,  # Adjust for creativity
        top_p=0.1,  # Nucleus sampling
        top_k=50,  # Top-k sampling
        no_repeat_ngram_size=2  # Prevent repeating n-grams
    )
    
    # Process the output to remove the input question from the response if it gets included
    answer = tokenizer.decode(output_sequences[0], skip_special_tokens=True).replace(prompt, "").strip()
    return answer

In [7]:
# Connect to the Qdrant database
from qdrant_client import models, QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams

client = QdrantClient(host='localhost', port=6333)

In [8]:
# Perform vector database search
def database_search(input_text):
    results = []
    hits = client.search(
        collection_name='movies',
        query_vector=filter_model.encode(input_text).tolist(),
        limit = 2
    )
    for idx, hit in enumerate(hits):
        result = {}
        result['original_title'] = hit.payload['original_title']
        result['genres'] = hit.payload['genres']
        result['overview'] = hit.payload['overview']
        results.append(result)
    return results

In [9]:
def answer_with_query(text, results):
    # Constructing a detailed prompt that structures the results as individual pieces of evidence
    evidence_list = "\n".join([f"Title: {result['original_title']}\nGenres: {result['genres']}\nOverview: {result['overview']}" for result in results])
    # prompt = f"Based on the following information, answer the question '{text}':\n\n{evidence_list}\n\nAnswer:"
    
    # answer = bot_response(prompt)
    return evidence_list

In [12]:
intentions = [
    "Can you recommend to me a movie similar to Harry Potter?",
    "What's the weather like today?",
    "Book a table for two at a Mexican restaurant tonight.",
    "Tell me a joke.",
    "How can I reset my password?",
    "Play some music from the 90s.",
    "What was the highest-grossing film in 2020?",
    "Find me a flight to New York next weekend.",
    "I need a recipe for a chocolate cake.",
    "What's the latest news on technology?",
    "Show me my photos from Paris.",
    "How do I make a Manhattan cocktail?",
    "Suggest a good workout for abs.",
    "What are some vegan restaurants nearby?",
    "Is there a good comedy movie to watch?",
    "I want to learn Python programming.",
    "Where can I watch The Office?",
    "Give me directions to the nearest gas station.",
    "What are the symptoms of COVID-19?",
    "Tell me about the life of Albert Einstein.",
    "Show me the latest horror films.",
    "What animated movies are recommended for kids?",
    "List all movies starring Tom Hanks.",
    "Who directed Pulp Fiction?",
    "Find movies similar to La La Land.",
    "What's the plot of The Matrix?",
    "Give me the soundtrack list from Guardians of the Galaxy.",
    "How many Oscars did Forrest Gump win?",
    "Is 'Joker' suitable for teenagers?",
    "What are some must-watch classic films?",
    "Play the trailer for 'Avengers: Infinity War'.",
    "I'm in the mood for a comedy. Any suggestions?",
    "What films are based on Stephen King's books?",
    "When was Schindler's List released?",
    "Find me a movie that will make me cry.",
    "What are the best sci-fi movies of the last decade?",
    "Which movies are in the Marvel Cinematic Universe?",
    "Tell me about the directorial style of Wes Anderson.",
    "How do I access movies on Netflix?",
    "Rate 'Mad Max: Fury Road' out of 10."
]

In [11]:
from torch.nn.functional import cosine_similarity

for intention in intentions:
    intention_vector = get_vector(intention)
    similarity = cosine_similarity(intention_vector.unsqueeze(0), valid_vector.unsqueeze(0))
    print(f"Question: {intention}")
    if similarity < 0.5:
        prompt = f"Answer the following question:\n{intention}"
        print(bot_response(prompt))
    if similarity >= 0.5:
        prompt = f"For the information mentioned, describe its genres and any notable themes:\n{intention}"
        bot_answer = bot_response(prompt)
        print(f"First answer:\n{bot_answer}")
        results = database_search(bot_answer)
        evidence_list = answer_with_query(intention, results)
        print(f"Databse search results:\n{evidence_list}")
        # print(f"Final answer:\n{answer}")
    print("============================================================")
        

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question: Can you recommend to me a movie similar to Harry Potter?


/home/nhannguyen/.local/share/virtualenvs/movie-recommendation-LLM-HHW9OGcc/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nhannguyen/.local/share/virtualenvs/movie-recommendation-LLM-HHW9OGcc/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


First answer:
I'm looking for a similar movie that has a magical world, a protagonist who is a wizard, and a plot that involves a quest for something. The movie should also have a sense of adventure and danger.
One movie you might like is The Chronicles of Narnia. It has similar thematic elements, such as the idea of a fantasy world and the importance of following one's heart. However, it is not a direct sequel to the Harry potter series, but it has its own unique story and characters.
Databse search results:
Title: The Wizard of Oz
Genres: Adventure, Family, Fantasy
Overview: Young Dorothy finds herself in a magical world where she makes friends with a lion, a scarecrow and a tin man as they make their way along the yellow brick road to talk with the Wizard and ask for the things they miss most in their lives. The Wicked Witch of the West is the only thing that could stop them.
Title: Конек-Горбунок
Genres: Adventure, Family, Fantasy
Overview: Adventures of Ivan the Fool and humpbacke

By this small test, we can see in the model answer well in general, however, there are a few questions the model does not response well. We would need to fine-tune it with the dataset wiki_movies from hugging face.